<a href="https://colab.research.google.com/github/sdikici/Demand_Forecasting_Prophet_DeepAR/blob/main/Prophet_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [2]:
!kaggle datasets download -d sercandikici/merged-dataset-electricty-weather-for-modelling
! unzip merged-dataset-electricty-weather-for-modelling.zip

  0% 0.00/189k [00:00<?, ?B/s]
100% 189k/189k [00:00<00:00, 9.48MB/s]
Archive:  merged-dataset-electricty-weather-for-modelling.zip
  inflating: merged_data.csv         


In [27]:
from prophet import Prophet
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [107]:
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true))
    return mape

def root_mean_squared_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    mse = np.mean((y_true - y_pred) ** 2)
    rmse = np.sqrt(mse)
    return rmse

def r_squared(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    mean_y_true = np.mean(y_true)
    ss_total = np.sum((y_true - mean_y_true) ** 2)
    ss_residual = np.sum((y_true - y_pred) ** 2)
    r2 = 1 - (ss_residual / ss_total)
    return r2


In [108]:
df_merged = pd.read_csv("merged_data.csv")
df_merged['settlement_date'] = pd.to_datetime(df_merged['settlement_date'])
df_merged["tsd"] = np.square(df_merged["tsd"])
df_merged

,settlement_date,tsd,is_holiday,temp
0,2021-01-01 00:00:00,28552.00,1.0,1.55
1,2021-01-01 02:00:00,26579.75,1.0,1.60
2,2021-01-01 04:00:00,25236.00,1.0,0.55
3,2021-01-01 06:00:00,25792.25,1.0,-0.35
4,2021-01-01 08:00:00,28162.75,1.0,-0.10
...,...,...,...,...
13136,2023-12-31 16:00:00,34006.00,0.0,7.60
13137,2023-12-31 18:00:00,32273.00,0.0,8.20
13138,2023-12-31 20:00:00,27456.50,0.0,8.45
13139,2023-12-31 22:00:00,24139.25,0.0,8.35


Model 1 predict 7 days in the future

In [109]:
df_model_1 = df_merged[["tsd","settlement_date"]]
df_model_1.columns=["y","ds"]
df_model_1

,y,ds
0,28552.00,2021-01-01 00:00:00
1,26579.75,2021-01-01 02:00:00
2,25236.00,2021-01-01 04:00:00
3,25792.25,2021-01-01 06:00:00
4,28162.75,2021-01-01 08:00:00
...,...,...
13136,34006.00,2023-12-31 16:00:00
13137,32273.00,2023-12-31 18:00:00
13138,27456.50,2023-12-31 20:00:00
13139,24139.25,2023-12-31 22:00:00


train test split

In [110]:
split_from = 90*12 #train test split is from 90days

train_data = df_model_1[:-split_from]
test_data = df_model_1[-split_from:]
print(train_data.shape)
print(test_data.shape)

(12061, 2)
(1080, 2)


train parameters

In [129]:
freq = "2H"
seasonality_prior_scale = 0.01
changepoint_prior_scale = 0.05
days_to_predict=7
periods = days_to_predict*12
mcmc_samples = 50

Train the model

In [ ]:
m = Prophet(mcmc_samples=mcmc_samples, changepoint_prior_scale=changepoint_prior_scale, seasonality_prior_scale=seasonality_prior_scale)
m.fit(train_data)

future_1 = m.make_future_dataframe(periods=periods , freq=freq)

DEBUG:cmdstanpy:cmd: /usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin info
cwd: None
DEBUG:cmdstanpy:input tempfile: /tmp/tmp9iiz5awu/osk3zfzq.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp9iiz5awu/xm48kwng.json
08:08:55 - cmdstanpy - INFO - CmdStan start processing
INFO:cmdstanpy:CmdStan start processing


chain 1 |          | 00:00 Status

chain 2 |          | 00:00 Status

chain 3 |          | 00:00 Status

chain 4 |          | 00:00 Status

DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: 1
DEBUG:cmdstanpy:idx 1
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'id=1', 'random', 'seed=48681', 'data', 'file=/tmp/tmp9iiz5awu/osk3zfzq.json', 'init=/tmp/tmp9iiz5awu/xm48kwng.json', 'output', 'file=/tmp/tmp9iiz5awu/prophet_modelj7ec5_r9/prophet_model-20240427080855_1.csv', 'method=sample', 'num_samples=25', 'num_warmup=25', 'algorithm=hmc', 'adapt', 'engaged=1']
DEBUG:cmdstanpy:running CmdStan, num_threads: 1
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'id=2', 'random', 'seed=48681', 'data', 'file=/tmp/tmp9iiz5awu/osk3zfzq.json', 'init=/tmp/tmp9iiz5awu/xm48kwng.json', 'output', 'file=/tmp/tmp9iiz5awu/prophet_modelj7ec5_r9/prophet_model-20240427080855_2.csv', 'method=sample', 'num_samples=25', 'num_warmup=25', 'algorithm=hmc', 'adapt', 'engaged=1']
DEBUG:cmdstanpy:idx 2
DEBUG:cmdstanp

In [ ]:
#create forecasts
forecast_1 = m.predict(future_1)
forecast_1.tail()

In [ ]:
from prophet.plot import plot_plotly, plot_components_plotly

plot_plotly(m, forecast_1)


In [ ]:
forecast_days = forecast_1[forecast_1["ds"] >= test_data["ds"].iloc[0]]
test_days = test_data[(test_data["ds"] >= test_data["ds"].iloc[0])&(test_data["ds"] <= forecast_days["ds"].iloc[-1])]

In [ ]:
def forecast_plot(forecast_days,test_days, days):

  fig, ax = plt.subplots(figsize=(14, 4))
  ax.plot(forecast_days['ds'], forecast_days['yhat'], label='Forecast', color='red')
  ax.plot(test_days['ds'], test_days['y'], label='Actual', color='blue')
  ax.set_xlabel('Date')
  ax.set_ylabel('MGW')
  plt.title(f'Prophet Forecast - {days} days')
  plt.legend()
  plt.show()

forecast_plot(forecast_days,test_days,days_to_predict)

In [ ]:
mape_1 = mean_absolute_percentage_error(test_days["y"],forecast_days["yhat"])
rmse_1 = root_mean_squared_error(test_days["y"],forecast_days["yhat"])
rsqr_1 = r_squared(test_days["y"],forecast_days["yhat"])
print(f"MAPE without regressors: {round(mape_1,7)}")
print(f"RMSE without regressors: {round(rmse_1,7)}")
print(f"R-squared without regressors: {round(rsqr_1,7)}")

training model_2, additional holidays

In [ ]:
m2 = Prophet(mcmc_samples=mcmc_samples, changepoint_prior_scale=changepoint_prior_scale, seasonality_prior_scale=seasonality_prior_scale)
m2.add_country_holidays(country_name="UK")
m2.fit(train_data)

future_2 = m2.make_future_dataframe(periods=periods , freq=freq)
forecast2 = m2.predict(future_2)
forecast2

In [ ]:
forecast2__days = forecast2[forecast2 ["ds"] >= test_data["ds"].iloc[0]]
forecast_plot(forecast2__days,test_days,days_to_predict)

In [ ]:
mape_2 = mean_absolute_percentage_error(test_days["y"],forecast2__days["yhat"])
rmse_2 = root_mean_squared_error(test_days["y"],forecast2__days["yhat"])
rsqr_2 = r_squared(test_days["y"],forecast2__days["yhat"])
print(f"MAPE holidays added: {round(mape_2,7)}")
print(f"RMSE holidays added: {round(rmse_2,7)}")
print(f"R-squared holidays added: {round(rsqr_2,7)}")

model 3 holiday+weather data

In [ ]:
df_model_3 = df_merged[["tsd","settlement_date","temp"]]
df_model_3.columns=["y","ds","temp"]
df_model_3

In [ ]:
train_data_3 = df_model_3[:-split_from]
test_data_3 = df_model_3[-split_from:]

In [ ]:
m3= Prophet(mcmc_samples=mcmc_samples, changepoint_prior_scale=changepoint_prior_scale, seasonality_prior_scale=seasonality_prior_scale)
m3.add_country_holidays(country_name="UK")
m3.add_regressor("temp", mode="additive")
m3.fit(train_data_3)

future3 = m3.make_future_dataframe(periods=periods, freq=freq)



In [ ]:
#create train and text idx which will be used to index future dataframe
train_idx = future3["ds"].isin(train_data_3.ds)
test_idx = ~train_idx

#add regressors to future dataframe

reg = ["temp"]
for r in reg:
  future3.loc[train_idx, r] = train_data_3[r].to_list()

for r in reg:
  future3.loc[test_idx, r] = test_data_3.iloc[:periods][r].to_list()

future3

In [ ]:
forecast3 = m3.predict(future3)
forecast3

In [ ]:
forecast3_days = forecast3[forecast3["ds"] >= test_data_3["ds"].iloc[0]]
test3_days = test_data_3[(test_data_3["ds"] >= test_data_3["ds"].iloc[0])&(test_data_3["ds"] <= forecast3_days["ds"].iloc[-1])]
forecast_plot(forecast3_days,test3_days,days_to_predict)

In [ ]:
mape_3 = mean_absolute_percentage_error(test3_days["y"],forecast3_days["yhat"])
rmse_3 = root_mean_squared_error(test3_days["y"],forecast3_days["yhat"])
rsqr_3 = r_squared(test3_days["y"],forecast3_days["yhat"])
print(f"MAPE holidays+weather added: {round(mape_3,7)}")
print(f"RMSE holidays+weather added: {round(rmse_3,7)}")
print(f"R-squared holidays+weather added: {round(rsqr_3,7)}")

In [ ]:
m3.plot_components(forecast3)